In [6]:
import numpy as np
 
np.set_printoptions(precision=3,suppress=True)
A_k_minus_1 = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])
 
process_noise_v_k_minus_1 = np.array([0.01,0.01,0.003])

Q_k = np.array([[1.0,0,0], [0, 1.0,   0],[0,0,1.0]])
                 

H_k = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])
                         

R_k = np.array([[1.0,0,0],[0,1.0,0],[0,0,1.0]])  

sensor_noise_w_k = np.array([0.07,0.07,0.04])
 
def getB(yaw, deltak):
    B = np.array([[np.cos(yaw)*deltak, 0],[np.sin(yaw)*deltak, 0],[0, deltak]])
    return B
 
def ekf(z_k_observation_vector, state_estimate_k_minus_1, control_vector_k_minus_1, P_k_minus_1, dk):
    state_estimate_k = A_k_minus_1 @ (state_estimate_k_minus_1) + (getB(state_estimate_k_minus_1[2],dk)) @ (control_vector_k_minus_1) + (process_noise_v_k_minus_1)       
    print(f'State Estimate Before EKF={state_estimate_k}')
    P_k = A_k_minus_1 @ P_k_minus_1 @ A_k_minus_1.T + (
            Q_k)
    measurement_residual_y_k = z_k_observation_vector - (
            (H_k @ state_estimate_k) + (
            sensor_noise_w_k))
    print(f'Observation={z_k_observation_vector}')          
    S_k = H_k @ P_k @ H_k.T + R_k
    K_k = P_k @ H_k.T @ np.linalg.pinv(S_k)
    state_estimate_k = state_estimate_k + (K_k @ measurement_residual_y_k)    
    P_k = P_k - (K_k @ H_k @ P_k)
    print(f'State Estimate After EKF={state_estimate_k}')
    return state_estimate_k, P_k
